# Notebook: Train Model for a given Condition

## Packages

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import random
import json

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 2000
N_SYNTH = 0
TARGET = "aspect_category"
LLM_SAMPLING = "fixed"

## Settings (do not change!)

In [3]:
N_FOLDS = 5

In [4]:
SPLIT_LOOP = [0, 1, 2, 3, 4, 0, 1, 2, 3]

In [5]:
RANDOM_SEED = 43
random.seed(RANDOM_SEED)

## Code

### Load Dataset

In [6]:
# Load Real Dataset
splits_real = []
for i in range(N_FOLDS):
    with open(f'../03 dataset split/real/real_{i}.json', 'r') as json_datei:
        real_split = json.load(json_datei)[:N_REAL]
        splits_real.append(real_split)

In [7]:
# Load Synth Dataset
splits_synth = []
for i in range(N_FOLDS):
    with open(f'../04 llm synthesis/synth/{LLM_NAME}/{LLM_SAMPLING}/split_{i}.json', 'r') as json_datei:
        synth_split = json.load(json_datei)[:N_SYNTH]
        splits_synth.append(synth_split)

In [8]:
n_splits_map = {
    500: 1,
    1000: 2,
    2000: 4
}
n_splits_required_real = n_splits_map.get(N_REAL, 0)
n_splits_required_synth = n_splits_map.get(N_SYNTH, 0)
n_splits_required_real, n_splits_required_synth

(4, 0)

In [9]:
# Five indexes, each for one cross valdiation run
train_dataset = []
test_dataset = []

for i in range(N_FOLDS):
    test_data = splits_real[i]
    train_data = []
    
    if N_REAL > 0:
        for split_idx in SPLIT_LOOP[i+1: i+1+n_splits_required_real]:
            for example in splits_real[split_idx]:
                train_data.append(example)
 
    if N_SYNTH > 0:
        for split_idx in SPLIT_LOOP[i+1: i+1+n_splits_required_synth]:
            for example in splits_synth[split_idx]:
                train_data.append(example)
                
    random.shuffle(train_data)
    
    train_dataset.append(train_data)
    test_dataset.append(test_data)

### Preprocessing

### Load Model

In [10]:
if TARGET == "aspect_category":
    model_name = "deepset/gbert-large"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

NameError: name 'DataCollatorWithPadding' is not defined